### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Create array of unique school names
unique_school_names = school_data_complete["school_name"].unique()

# Finds number of schools from length of unique school names array
school_count = len(unique_school_names)

# Calculates student count from length of student names
student_count = len(school_data_complete["student_name"])

# Calculates total budget size from unique district budgets
total_budget = school_data_complete["budget"].unique()
total_budget = total_budget.sum()

# Calculates the average math score
avg_math_score = school_data_complete["math_score"].mean()

# Calculates the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()

# Calculate % of students that pass math
math_pass = school_data_complete.loc[school_data_complete["math_score"]>=70]["math_score"].count()
math_pass_percentage = (math_pass/student_count)

# Calculate % of students that pass reading
reading_pass = school_data_complete.loc[school_data_complete["reading_score"]>=70]["reading_score"].count()
reading_pass_percentage = (reading_pass/student_count)

# Calculate % of students that pass both math & reading
math_reading_pass = school_data_complete.loc[(school_data_complete["math_score"]>=70) 
                                             & (school_data_complete["reading_score"]>=70)]["student_name"].count()/student_count


# Create Data Frame to hold results
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                "Total Students": [student_count],
                                "Total Budget": [total_budget],                                
                                "Average Math Score": [avg_math_score],
                                "Average Reading Score": [avg_reading_score],
                                "% Passing Math":[math_pass_percentage],
                                "% Passing Reading": [reading_pass_percentage],
                                "Overall Passing Rate": [math_reading_pass]})

# Format for Data Types
district_summary.style.format({"Total Budget": "${:,.0f}", 
                               "Average Math Score": "{:.1f}%",
                               "Average Reading Score": "{:.1f}%",
                               "% Passing Math": "{:.1%}", 
                               "% Passing Reading": "{:.1%}", 
                               "Overall Passing Rate": "{:.1%}"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428",79.0%,81.9%,75.0%,85.8%,65.2%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Group by school name
school_group = school_data_complete.set_index("school_name").groupby(["school_name"])

# Create School Types 
#school_types = school_data_complete.set_index('school_name')['type'].groupby(["school_name"])
school_types = school_group["type"].unique().astype("str")

# Total Students per School
students_per_school = school_group["Student ID"].count()

# Total School Budget
#school_budget = school_data_complete.set_index('school_name')['budget']
school_budget = school_group["budget"].sum()

# Per Student Budget
student_budget = school_data_complete.set_index("school_name")["budget"]/ school_data_complete.set_index("school_name")["size"]
student_budget = student_budget.groupby("school_name").unique().astype("float")

# Average Scores
avg_math = school_group["math_score"].mean()
avg_read = school_group["reading_score"].mean()

# % of Overall Passing
pass_math = school_data_complete[school_data_complete["math_score"]>=70].groupby("school_name")["Student ID"].count()/students_per_school
pass_read = school_data_complete[school_data_complete["reading_score"]>=70].groupby("school_name")["Student ID"].count()/students_per_school
overall = school_data_complete[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70)].groupby("school_name")["Student ID"].count()/students_per_school

# Create Data Frame 
school_summary = pd.DataFrame({"School Type": school_types,
                               "Total Students": students_per_school,
                               "Per Student Budget": student_budget,
                               "Total School Budget": school_budget,
                               "Average Math Score": avg_math,
                               "Average Reading Score": avg_read,
                               "% Passing Math": pass_math,
                               "% Passing Reading": pass_read,
                               "Overall Passing Rate": overall})

# Give Index name "School Name"
school_summary.index.name = "School Name"

# Format Summary
school_summary.style.format({"Total Students": "{:,}", 
                             "Total School Budget": "${:,.0f}", 
                             "Per Student Budget": "${:,.0f}",
                             "Average Math Score": "{:.1f}", 
                             "Average Reading Score": "{:.1f}", 
                             "% Passing Math": "{:.1%}", 
                             "% Passing Reading": "{:.1%}", 
                             "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,['District'],"4,976",$628,"$15,549,641,728",77.0,81.0,66.7%,81.9%,54.6%
Cabrera High School,['Charter'],"1,858",$582,"$2,009,159,448",83.1,84.0,94.1%,97.0%,91.3%
Figueroa High School,['District'],"2,949",$639,"$5,557,128,039",76.7,81.2,66.0%,80.7%,53.2%
Ford High School,['District'],"2,739",$644,"$4,831,365,924",77.1,80.7,68.3%,79.3%,54.3%
Griffin High School,['Charter'],"1,468",$625,"$1,346,890,000",83.4,83.8,93.4%,97.1%,90.6%
Hernandez High School,['District'],"4,635",$652,"$14,007,062,700",77.3,80.9,66.8%,80.9%,53.5%
Holden High School,['Charter'],427,$581,"$105,933,149",83.8,83.8,92.5%,96.3%,89.2%
Huang High School,['District'],"2,917",$655,"$5,573,322,295",76.6,81.2,65.7%,81.3%,53.5%
Johnson High School,['District'],"4,761",$650,"$14,733,628,650",77.1,81.0,66.1%,81.2%,53.5%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# Create Series based on the sorted overall pass %, and display top 5 best first
top_5 = pd.DataFrame({"Overall Passing Rate": overall})

top_5 = top_5.sort_values("Overall Passing Rate",ascending = False)

# Create Name "School Name" for Index
top_5.index.name="School Name"

top_5.head().style.format({"Overall Passing Rate": "{:.1%}"})

,Overall Passing Rate
School Name,
Cabrera High School,91.3%
Thomas High School,90.9%
Griffin High School,90.6%
Wilson High School,90.6%
Pena High School,90.5%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# Uses previous series and display bottom 5 sorted to worst first
bottom_5 = pd.DataFrame({"Overall Passing Rate": overall}) 
bottom_5 = bottom_5.sort_values("Overall Passing Rate")

# Create Name "School Name" for Index
bottom_5.index.name="School Name"


# Format for %
bottom_5.head().style.format({"Overall Passing Rate": "{:.1%}"})



,Overall Passing Rate
School Name,
Rodriguez High School,53.0%
Figueroa High School,53.2%
Huang High School,53.5%
Hernandez High School,53.5%
Johnson High School,53.5%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# Create table displaying math scores based on grade level
ninth_math = school_data_complete.loc[school_data_complete["grade"] =="9th"].groupby("school_name")["math_score"].mean()
tenth_math = school_data_complete.loc[school_data_complete["grade"] =="10th"].groupby("school_name")["math_score"].mean()
eleventh_math = school_data_complete.loc[school_data_complete["grade"] =="11th"].groupby("school_name")["math_score"].mean()
twelfth_math = school_data_complete.loc[school_data_complete["grade"] =="12th"].groupby("school_name")["math_score"].mean()

math_scores = pd.DataFrame({"9th": ninth_math,
                            "10th": tenth_math,
                            "11th": eleventh_math,
                            "12th": twelfth_math})

math_scores = math_scores[["9th","10th","11th","12th"]]
math_scores.index.name = "School Name"

# Format
math_scores.style.format({"9th": "{:.1f}", 
                          "10th": "{:.1f}", 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Create table displaying math scores based on grade level
ninth_read = school_data_complete.loc[school_data_complete["grade"] =="9th"].groupby("school_name")["reading_score"].mean()
tenth_read = school_data_complete.loc[school_data_complete["grade"] =="10th"].groupby("school_name")["reading_score"].mean()
eleventh_read = school_data_complete.loc[school_data_complete["grade"] =="11th"].groupby("school_name")["reading_score"].mean()
twelfth_read = school_data_complete.loc[school_data_complete["grade"] =="12th"].groupby("school_name")["reading_score"].mean()

read_scores = pd.DataFrame({"9th": ninth_math,
                            "10th": tenth_math,
                            "11th": eleventh_math,
                            "12th": twelfth_math})

read_scores = read_scores[["9th","10th","11th","12th"]]
read_scores.index.name = "School Name"

# Format
read_scores.style.format({"9th": "{:.1f}", 
                          "10th": "{:.1f}", 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [31]:
# Create Bins based on Spending
bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ["< $585", "$585 - 614", "$615 - 644", "> $644"]
school_data_complete["spending_bins"] = pd.cut(school_data_complete["budget"]/school_data_complete["size"],bins,labels = group_name)

# Group by Spending Bins
spending_group = school_data_complete.groupby("spending_bins")

# Calculations based on School Spending
avg_math = spending_group["math_score"].mean()
avg_read = spending_group["reading_score"].mean()
pass_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("spending_bins")["Student ID"].count()/spending_group['Student ID'].count()
pass_read = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("spending_bins")["Student ID"].count()/spending_group["Student ID"].count()
overall = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("spending_bins")["Student ID"].count()/spending_group["Student ID"].count()

# Create new Data Frame
scores_by_spending = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "% Passing Math": pass_math,
    "% Passing Reading": pass_read,
    "Overall Passing Rate": overall}) 

# Give a new index and name it Per Student Budget
scores_by_spending.index.name = "Per Student Budget"
scores_by_spending = scores_by_spending.reindex(group_name)

# Format for Data Types
scores_by_spending.style.format({"Average Math Score": "{:.1f}", 
                                 "Average Reading Score": "{:.1f}", 
                                 "% Passing Math": "{:.1%}", 
                                 "% Passing Reading":"{:.1%}", 
                                 "Overall Passing Rate": "{:.1%}"})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
< $585,83.4,84.0,93.7%,96.7%,90.6%
$585 - 614,83.5,83.8,94.1%,95.9%,90.1%
$615 - 644,78.1,81.4,71.4%,83.6%,60.3%
> $644,77.0,81.0,66.2%,81.1%,53.5%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Create Bins based on School Size
bins = [0, 499,999,1499, 1999,2499,2999, 99999999999]
group_name = ["<500", "500-999","1000-1499" ,"1500-1999","2000-2499" , "2500-2999", ">=3000)"]
school_data_complete["size_bins"] = pd.cut(school_data_complete["size"], bins, labels = group_name)

# Group by Spending Bins
by_size = school_data_complete.groupby("size_bins")

# Calculations based on School Size
avg_math = by_size["math_score"].mean()
avg_read = by_size["reading_score"].mean()
pass_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("size_bins")['Student ID'].count()/by_size["Student ID"].count()
pass_read = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("size_bins")['Student ID'].count()/by_size["Student ID"].count()
overall = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("size_bins")["Student ID"].count()/by_size["Student ID"].count()
            
# Create new Data Frame          
scores_by_size = pd.DataFrame({"Average Math Score": avg_math,
                               "Average Reading Score": avg_read,
                               "% Passing Math": pass_math,
                               "% Passing Reading": pass_read,
                               "Overall Passing Rate": overall})
                
# Give a new index and name it Total Students
scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)

# Format by Data Types
scores_by_size.style.format({"Average Math Score": "{:.1f}", 
                             "Average Reading Score": "{:.1f}", 
                             "% Passing Math": "{:.1%}", 
                             "% Passing Reading":"{:.1%}", 
                             "Overall Passing Rate": "{:.1%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
<500,83.8,83.8,92.5%,96.3%,89.2%
500-999,83.8,84.0,94.6%,95.9%,90.5%
1000-1499,83.4,83.8,93.4%,97.1%,90.6%
1500-1999,83.4,83.9,93.7%,96.7%,90.6%
2000-2499,83.3,84.0,93.9%,96.5%,90.6%
2500-2999,76.8,81.0,66.6%,80.5%,53.7%
>=3000),77.1,80.9,66.5%,81.1%,53.7%


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
# Group by School Types "District" and "Charter"
by_type = school_data_complete.groupby("type")

# Calcualtions for Type of School Group
avg_math = by_type["math_score"].mean()
avg_read = by_type["reading_score"].mean()
pass_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("type")["Student ID"].count()/by_type["Student ID"].count()
pass_read = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("type")["Student ID"].count()/by_type["Student ID"].count()
overall = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("type")["Student ID"].count()/by_type["Student ID"].count()

# df build            
scores_by_type = pd.DataFrame({"Average Math Score": avg_math,
                               "Average Reading Score": avg_read,
                               "% Passing Math": pass_math,
                               "% Passing Reading": pass_read,
                               "Overall Passing Rate": overall})
    
# Rename the index to "Type of School"
scores_by_type.index.name = "Type of School"

# Format for Data Types
scores_by_type.style.format({"Average Math Score": "{:.1f}", 
                             "Average Reading Score": "{:.1f}", 
                             "% Passing Math": "{:.1%}", 
                             "% Passing Reading":"{:.1%}", 
                             "Overall Passing Rate": "{:.1%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.9,93.7%,96.6%,90.6%
District,77.0,81.0,66.5%,80.9%,53.7%


## My Data Frame for Analysis Overview

In [51]:
# Create Bins based on School Size
bins = [0, 499,999,1499, 1999,2499,2999, 99999999999]
group_name = ["<500", "500-999","1000-1499" ,"1500-1999","2000-2499" , "2500-2999", ">=3000)"]
school_data_complete["size_bins"] = pd.cut(school_data_complete["size"], bins, labels = group_name)

# Group by Spending Bins
by_size = school_data_complete.groupby("size_bins")

# Count of district type
charter_count = school_data_complete[school_data_complete["type"] == "Charter"].groupby("size_bins")["type"].count().astype("int")
district_count = school_data_complete[school_data_complete["type"] == "District"].groupby("size_bins")["type"].count().astype("int")


# Calculations based on School Size
student_budget = by_size["budget"].mean()/by_size["Student ID"].count()
student_budget.head()

pass_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("size_bins")["Student ID"].count()/by_size["Student ID"].count()
pass_read = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("size_bins")["Student ID"].count()/by_size["Student ID"].count()
overall = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].groupby("size_bins")["Student ID"].count()/by_size['Student ID'].count()
            

# Create new Data Frame          
scores_by_size = pd.DataFrame({"Number of Charter": charter_count,
                               "Number of District": district_count,
                               "Average Per Student Budget":student_budget,
                               "% Passing Math": pass_math,
                               "% Passing Reading": pass_read,
                               "Overall Passing Rate": overall})
            
# Give a new index and name it Total Students
scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)

# Format by Data Types
scores_by_size.style.format({"Average Per Student Budget":"${:,.0f}",
                             "% Passing Math": "{:.1%}", 
                             "% Passing Reading":"{:.1%}", 
                             "Overall Passing Rate": "{:.1%}"})

,Number of Charter,Number of District,Average Per Student Budget,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,,
<500,427,0,$581,92.5%,96.3%,89.2%
500-999,962,0,$609,94.6%,95.9%,90.5%
1000-1499,1468,0,$625,93.4%,97.1%,90.6%
1500-1999,7054,0,$150,93.7%,96.7%,90.6%
2000-2499,2283,0,$578,93.9%,96.5%,90.6%
2500-2999,0,8605,$216,66.6%,80.5%,53.7%
>=3000),0,18371,$161,66.5%,81.1%,53.7%


## Summary of Data

In [ ]:
"""
#1 Public Districts are failing at a much higher rate than private charters, statistically 1/2 of all students going to a public school district are going to fail either math or reading while 90% of their peers are passing at Private Charter Schools
#2 The Data initially suggests that Public Districts are underfunded, however, the lowest funded group are mid-range sized Private Charters. These Charter Schools are the lowest funded group and tied for #1 overall performace suggesting their is a 
sweet spot in which the class size lowers overhead/student but still allows for top notch performance scores.
#3 Reading scores are better compared to math in every Size and District suggesting that the curriculum for teaching math needs to be re-evaluated across the board. Another analysis could be performed fours after a new curriculum is implemented for math 
to determine if it is more or less effective relative to both old curriculum and the reading curriculum."""